In [ ]:
import pandas as pd
import numpy as np
import warnings
from sklearn.feature_selection import VarianceThreshold
warnings.filterwarnings("ignore")
pd.options.display.max_columns = None
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (16,7)
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve,roc_auc_score,precision_score,recall_score
from sklearn.ensemble import RandomForestClassifier
from scikitplot.helpers import binary_ks_curve
from matplotlib import pyplot
from sklearn.model_selection import validation_curve
from category_encoders import OneHotEncoder,TargetEncoder
from yellowbrick.model_selection import RFECV
import shap

### Read dataframe

In [ ]:
df_train = pd.read_csv("../input/santander-customer-satisfaction/train.csv")
df_train.head()

## Defining features

Aqui estamos dividindo o dataframe em 3 grupos:

#### 1 - Colunas de identificação

- Colunas que representam chaves ID 

#### 2 Target

- Coluna que identifica a target do problema

#### 3 Variáveis dependentes/explicativas

- Aqui são todas as features disponiveis, esse grupo foi dividido em 2 subgrupos que são variaveis categoricas e variaveis continuas

In [ ]:
id_columns = ['ID']
target_column = ['TARGET']

num_vars = df_train.select_dtypes(include=['float64','int64'])
cat_vars = df_train.select_dtypes(include=['object'])

print('initial numerical vars =',len(num_vars.columns))
print('initial categorical vars =',len(cat_vars.columns))

y = df_train[target_column]

### Dropping null features if exists

In [ ]:
def drop_nulls(df,threshold,num_features):
    missing = pd.DataFrame({'types':df.filter(num_features).dtypes, 'percentual_nulo': df.filter(num_features).isna().sum()/len(df.filter(num_features))})
    missing = missing[missing['percentual_nulo'] > threshold].sort_values(by='percentual_nulo',ascending=False)
    print(missing)
    deletar = list(missing[missing['percentual_nulo'] > threshold].index)
    print('number of null features = ',len(deletar))
    df_drop = df.drop(columns= deletar,axis=1)
    return df_drop

df_train_null = drop_nulls(df_train,0.5,num_features = num_vars )

### Dimensionality reduction (Variance threshould, multi correlation)

Com intuito de minimizar o custo de altas dimensões, ajudando a diminuir a complexidade computacional de todos algoritmos que serão criados e diminuir a chance de problemas de sobreajuste devido ao alto numero de dimensoes iremos fazer uma análise para diminuir o espaco de possibilidades. Todo processo foi divididos nessas etapas:

- Eliminação de variaveis constantes ou que possuam pouca variação (variance threshould)
- Eliminação de multicolinearidade em variaveis continuas
- Eliminação de variaveis categoricas com mais de 10 níveis 

### Número total de features disponíveis 

In [ ]:
print('Número total de features =', len(df_train_null.columns))

### Drop categorical feature with many values

In [ ]:
def drop_categorical_with_many_values(df,categorical_features, threshold):
    df.filter(cat_vars).nunique().sort_values(ascending=False).head(50).plot(kind='bar')
    condition = df.filter(cat_vars).nunique().sort_values(ascending=False).reset_index()[0]>threshold
    list_to_drop = df.filter(cat_vars).nunique().sort_values(ascending=False).reset_index()[condition]
    print('number of feauters to delete = ',len(list_to_drop))
    df_train_null_drop = df.drop(columns = list_to_drop['index'])
    return df_train_null_drop

# df_train_null_drop = drop_categorical_with_many_values(df = df_train_null
#                                                        ,categorical_features = cat_vars
#                                                        , threshold = 10)

### Number of features after treatment

In [ ]:
df_train_null_drop = df_train_null.copy()
print('Número total de features =', len(df_train_null_drop.columns))

### Dropping duplicates columns

In [ ]:
# Checking if there is any duplicated column
remove = []
cols = df_train_null_drop.columns
for i in range(len(cols)-1):
    column = df_train_null_drop[cols[i]].values
    for j in range(i+1,len(cols)):
        if np.array_equal(column, df_train_null_drop[cols[j]].values):
            remove.append(cols[j])


# If yes, than they will be dropped here
df_train_null_drop = df_train_null_drop.drop(remove, axis = 1)
print('Número total de features =', len(df_train_null_drop.columns))

### Split in train and test

- Foram usados 25% dos dados para teste e 75% para treino
- O split foi feito de forma estratificada para preservarmos a distribuição da target em treino e teste

In [ ]:
x = df_train_null_drop.fillna(0)


#Split em treino e teste 
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,random_state=42,stratify=y)

print("number of rows in train data = ",len(x_train))
print('---------------------------')
print(y_train.value_counts().reset_index())
print('---------------------------')
print("number of rows in test data = ",len(x_test))
print('---------------------------')
print(y_test.value_counts().reset_index())

## Defining some function (Outliers treatment, variance threshould and multi correlation)

### Clip de outliers
- Para usarmos as técnicas de agrupamento fizemos o tratamento de outliers, pois o algoritmo escolhido para clusterização é sensivel a dados ruidos

### Variance threshould

- Foi utilizado o variance threshould nas variaveis continuas para tirar variaveis constantes ou que possuam uma pequena variação essa estratégia tem o intuito de não deixar que pequenas variações impactem no modelo, reduzindo a chance de um sobreajuste/overfitting e além disso diminuir o número de dimensões para diminuirmos o custo computacional dos próximos passos

### Eliminação de features correlacionadas

- Foi retirado a correlação entre as variaveis continuas para que possamos diminuir o custo computacional dos próximos passos

In [ ]:
def clip_outliers(df,drop_columns,q_min,q_max):
    
    num_vars_name = df.select_dtypes(include=['float64']).columns     
    for i in num_vars_name:
        
        #get max and min quantile
        min_value = df.loc[:,i].quantile(q_min)
        max_value = df.loc[:,i].quantile(q_max)

        #replace values with max and min quantile value
        df.loc[:,i] = np.where(df.loc[:,i] < min_value, min_value,df.loc[:,i])
        df.loc[:,i] = np.where(df.loc[:,i] > max_value, min_value,df.loc[:,i])
        
    return df

def variance_threshold(df,threshold):
    vt = VarianceThreshold(threshold=threshold)

    vt.fit(df)

    mask = vt.get_support()

    num_vars_reduced = df.iloc[:, mask]
    return num_vars_reduced

def correlation(df, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in df.columns:
                    del df[colname] # deleting the column from the dataset

    return df

### Aplplying variance threshould, drop correlation features and clip the outliers

In [ ]:
num_vars_vt = variance_threshold(x_train.filter(num_vars),threshold = 0.01)
num_vars_vt_corr = correlation(x_train.filter(num_vars_vt), threshold = 0.7)

#Scaler feature for clustering analyses
scaler = StandardScaler()
scaler.fit(x_train.filter(num_vars_vt_corr))

#Apply in train and test
x_train.loc[:,num_vars_vt_corr.columns] = scaler.transform(x_train.loc[:,num_vars_vt_corr.columns])
x_test.loc[:,num_vars_vt_corr.columns] = scaler.transform(x_test.loc[:,num_vars_vt_corr.columns])

           
#Select important features
x_train = x_train.filter(list(num_vars_vt_corr.columns)+list(cat_vars)+list(id_columns)).fillna(0)
x_test  =  x_test.filter(list(num_vars_vt_corr.columns)+list(cat_vars)+list(id_columns)).fillna(0)


#Reduce outliers for clustering analyses
x_train = clip_outliers(x_train,drop_columns=id_columns,q_min=0.05,q_max=0.95)
x_test = clip_outliers(x_test,drop_columns=id_columns,q_min=0.05,q_max=0.95)


print('Número total de features =', len(x_train.columns))

### One hot enconder

- Aqui todas as variáveis categoricas que sobraram do step de retirada de cardionalidade serão transformadas em dummy 0 e 1

In [ ]:
### Convert categoricals features to string
# d = dict.fromkeys(x_train.select_dtypes(np.int64).columns, 'str')
# x_train =x_train.astype(d)
# x_test = x_test.astype(d)

# ##Fitting a oneHotEnconder
# enc = OneHotEncoder().fit(x_train.drop(columns=id_columns))

# x_train  = enc.transform(x_train.drop(columns=id_columns))
# x_test = enc.transform(x_test.drop(columns=id_columns))

print(x_train.shape,x_test.shape)

### Recursive Feature elimination (Feature selection)

Com intuito de selecionar as variaveis que melhor irão nos ajudar a discriminar nosso problema iremos utilizar um método chamado recursive feature elimination. esse método funciona da seguinte forma:

- 1) Treina um modelo com todas as features
- 2) Elimina as features com feature_importances_ menores
- 3) Retreina um novo modelo com as features restantes
- 4) Repete passo 2 e 3
- 5) Avalia o número de features selecionadas versus a métrica de sucesso do seu modelo

Aqui escolhemos o algoritmo de Random forest e a métrica de avaliação a ROC AUC

In [ ]:
fs_model = RandomForestClassifier(max_depth=6,n_jobs=-1,n_estimators=100,class_weight='balanced')

# Instantiate RFECV visualizer with a linear Random forest classifier
visualizer = RFECV(fs_model,scoring='roc_auc',cv=3,step=0.1)

# Fit the data to the visualizer
visualizer.fit(x_train, y_train) 

# Finalize and render the figure
visualizer.show()           

### Select the best features

In [ ]:
print('Optimal number of features :', visualizer.n_features_)
best_features = list(x_train.columns[visualizer.support_])
print('features selecionadas: ', best_features)

x_train = x_train[best_features]
x_test = x_test[best_features]

### Clustering analysis

O objetivo dessa frente é segmentar o conjunto de dados, encontrando padrões escondidos na nossa base, a ideia é que pontos dentro do mesmo grupo sejam parecidos entre si **(coesão interna)** e que pontos de grupos distintos sejam diferentes **(separabilidade)**

Para conseguirmos esse tipo de agrupamento podemos utilizar 3 tipos de agrupamento que são:

- Particionais
- hierarquicos
- Baseado em densidade

Aqui escolhemos um método particional que é o algoritmo de kmeans que funcionará da seguinte forma:

- 1) Escolha K centroides
- 2) Atribua cada ponto ao K centroide mais próximo
- 3) Recalcula o centro do centroide com base na média dos pontos do cluster
- 4) Repete-se o passo 2 e 3 até que se atinja um número pré definido de iterações ou um limiar mínimo de mudanças no centroide que mudem a função de custo

Um dos problemas desse tipo de agrupamento particional é que temos que definir (antes de rodar) o número ideial de clusters e como não sabemos o número ideal de clusters que melhor irar separar os dados iremos fazer algumas iterações para verificar o número de K ótimo

### Defining the best number of K (cluster)

Para definirmos o melhor número de clusters para o nosso agrupamento vamos utilizar 3 métricas que são:

**1) Método Elbow**

Elbow é um método visual para estimarmos o número ótimo de clusters, basicamente rodamos o Kmeans multiplas vezes para K diferentes e plotamos o inertia **(soma das distancias ao quadrado de cada ponto ao seu respectivo cluster)**.
O único problema desse método é que estamos olhando apenas o conceito de **coesa interna**, ou seja, estamos escolhendo um K que de certa forma minimiza a distancia intra cluster, proem não estamos olhando o quesito de **separabilidade** (máximizar a distancia entre clusters)

**2) Silhouette Score**

Nesse método iremos usar os dois conceitos mencionados acima coesao e separabilidade, por fim queremos um cluster que minimize as distancias intra clusters (coesao) e maximize a distancia entre clusters (separbilidade). O silhouete score trabalha com esses dois conceitos, utilizando a propria inertia calculada pela funcao de custo do kmeans e a distancia entre o ponto e os pontos do clusters mais proximo ao seu. a formula fica:

a = distancia intra cluster / inertia 
b = distancia entre cluster (vizinho mais próximo)

silhouette = (b - a) / Max(a,b)

Um ponto de atenção é que para clusters diferentes de esféricos e com caracteristicas de densidade a silhouette perde o sentido, pois tem forte relação a métrica de distancia usada para calcular o A e B e no geral usa-se a distância euclidiana que gera clusters globulares

**3) Davies Bouldin score**

O método de Davies bouldin é baseado na razão entre distancia intra cluster e distancia entre clusters. A diferença emrelação ao silhouette é que esse método constuma ser mais custoso por considerar a distancia entre clusters e não só a distancia entre o cluster vizinho mais próximo 


### Kmeans with elbow method and silhouette score

- Aqui foi testado o kmeans direto com sua função de custo para termos uma ideia inicia de número cluster ótimo

In [ ]:
min = 3
max = 8
wcss = []
silhouette= []

train_kmeans = x_train.select_dtypes(include="float64")

for i in range(min, max):
    
    ##Training a kmeans model
    model = KMeans(n_clusters = i, random_state = 42)
    model.fit(train_kmeans)
    
    #Scoring
    pred = model.predict(train_kmeans)
    
    #Get silhouette score
    score = silhouette_score(train_kmeans, pred)
    
    # inertia method returns wcss for that model
    wcss.append(model.inertia_)
    print('Silhouette Score for k = {}: {:<.3f}'.format(i, score))
    
plt.figure(figsize=(10,max))
sns.lineplot(range(min, max), wcss,marker='o',color='red')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

### Kmeans with hyperopt to optimize silhouette score

- Aqui será ajustado a função de custo do hyperopt, mas inicialmente foi utilizado o silhouette score para termos clusters coesos e com alta separabilidade

In [ ]:
from hyperopt import fmin, tpe, hp
def objective(params):
    params = {'n_clusters': int(params['n_clusters'])
              ,'n_init': int(params['n_init'])
              ,'max_iter': int(params['max_iter'])}
    
    model = KMeans(random_state = 42, **params).fit(train_kmeans)
    pred = model.predict(train_kmeans)
    silhouette = silhouette_score(train_kmeans, pred)
    score = (1-silhouette)
    print("Silhouette {:.3f} params {}".format(silhouette , params))
    return score

space = {
    'n_clusters': hp.quniform('n_clusters', 3, 10, 1),
    'n_init': hp.quniform('max_depth', 10, 30, 2),
    "max_iter": hp.quniform('max_iter', 300, 1000, 50)
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=5)

### Testing Gaussian mix models

Aqui iremos testar um outro método de clusterização que é o GMM (Gaussian mixture models), é um modelo que tem como arte solucionar um dos problemas do Kmeans que só detectam clusters **globulares**, o GMM é uma generalização do kmeans, porem ao inves de estimar os centroides iremos estimar **3 parametros** que são:
- **Média**
- **Matriz de covariancia**
- **pesos das gausianas** (ou probabilidade de um ponto tenha sido gerado por uma gausiana k)

O algoritmo simular ao kmeans utiliza-se de **4 grandes passos**:

- 1) Inicialização dos parametros (média, covariancia e pesos)
- 2) Expectation (Calcula o valor esperado para a log da verossimelhanca com base em variável latente)
- 3) Maximization (Recalcula os paramétros com intuito de maximizar a log da verossimelhança)
- 4) Repete-se o passo 2 e 3 até que se atinja um número pré definido de iterações ou um limiar mínimo de mudanças na log verossimilhanca

Diferente do Kmeans onde a saída é uma partição rigida, aqui temos um modelo cujo a **saida é probabilistica**, então temos uma probabilidade a um ponto X pertencer a um cluster K, alem disso, devido a estimativa da matriz de covariancia o GMM produz clusters com diferentes formatos como elipses, fato que não ocorre no Kmeans. Outro fato interessante é a **função de custo** do GMM está relacionada a maximização da log verossimilhança, ou seja, queremos que aproximizar a distribuição das gausianas da distribuição final que gerou os dados. Apesar de todas as vantagens o GMM acaba sendo **mais custoso computacionalmente**, pois nas etapas 2 e 3 (EM) é necessário calcular a matriz inversa de covariancia o que deixa o modelo com uma complexidade O(K.N.D³).

**Principais parâmetros**

Além dos parâmetros conhecidos do kmeans, aqui temos o **covariance type** que irá ditar o formato dos nossos clusters com as opções full, tied, diag e spherical. O **init params** que está relacionado a inicialização dos 3 parâmetros que pode ser randomico ou até pelo proprio kmeans

**CUIDADO** - Caso tenhamos clusters diferentes de globulares no gmm, métricas como silhouette perdem o sentido

In [ ]:
from sklearn.mixture import GaussianMixture

min = 3
max = 8
silhouette= []

train_gmm = x_train.select_dtypes(include="float64")

for i in range(min,max):
    gmm = GaussianMixture(n_components = i, random_state = 42).fit(train_gmm) 
    pred=gmm.predict(train_gmm)
    score = silhouette_score(train_gmm, pred)
    print('Silhouette Score for k = {}: {:<.3f}'.format(i, score))

### Testing the same pipeline of hyperopt in GMM

Intuito de encontrar a maior silhouetta interandoa alguns hiperparametros para os proximos passos iremos iterar no covariance type para verificarmos se temos um formato melhor de covariancia que se adequa aos dados e iremos iterar a inicialização entre randomica ou iniciar com a saída de um kmeans

In [ ]:
from sklearn.mixture import GaussianMixture
from hyperopt import fmin, tpe, hp
def objective(params):
    params = {'n_components': int(params['n_components'])
              ,'n_init': int(params['n_init'])
              ,'max_iter': int(params['max_iter'])}

    model = GaussianMixture(random_state = 42, **params).fit(train_gmm) 
    pred = model.predict(train_gmm)
    silhouette = silhouette_score(train_gmm, pred)
    score = (1-silhouette)
    print("Silhouette {:.3f} params {}".format(silhouette , params))
    return score

space = {
    'n_components': hp.quniform('n_components', 3, 10, 1),
    'n_init': hp.quniform('max_depth', 10, 30, 2),
    'max_iter': hp.quniform('max_iter', 300, 1000, 50),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=5)

### InterclusterDistance 

É usado para termos de forma visual a separação de clusters por traz desse método é rodado um PCA com 2 componentes e por fim é plotado esses dois componentes em volta dos K clusters que definirmos

In [ ]:
from yellowbrick.cluster import InterclusterDistance
model = KMeans(n_clusters = 4 , random_state = 42)
model.fit(train_kmeans)

# Instantiate the clustering model and visualizer
visualizer = InterclusterDistance(model)

visualizer.fit(train_kmeans)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

In [ ]:
#Scoring
x_train['cluster'] = model.predict(x_train.filter(train_kmeans.columns))
x_test['cluster'] = model.predict(x_test.filter(train_kmeans.columns))

### Validation curve to diagnostic bias and variance in hiperparameters

Para implementarmos um modelo de classificação iremos investigar como a mudança de um parametro altera a performance do modelo, com isso iremos ter um range melhor para inserirmos nas buscas otimizadas. Além disso, essemétodo nos ajudará enxergar a relação vies e variancia de cada hiperparametro que iremos otimizar

In [ ]:
def plot_validation_curve(x,y,modelo,parametro,param_range,metrica):

    # Calculate accuracy on training and test set using range of parameter values
    train_scores, test_scores = validation_curve(modelo, 
                                                 x, 
                                                 y, 
                                                 param_name=parametro, 
                                                 param_range=param_range,
                                                 cv=3, 
                                                 scoring=metrica, 
                                                 n_jobs=-1)


    # Calculate mean and standard deviation for training set scores
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)

    # Calculate mean and standard deviation for test set scores
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    # Plot mean accuracy scores for training and test sets
    plt.plot(param_range, train_mean, label="Training score", color="red")
    plt.plot(param_range, test_mean, label="Cross-validation score", color="blue")

    # Plot accurancy bands for training and test sets
    plt.fill_between(param_range, train_mean - train_std, train_mean + train_std, color="gray")
    plt.fill_between(param_range, test_mean - test_std, test_mean + test_std, color="gray")

    # Create plot
    plt.title("Validation Curve With Random Forest")
    plt.xlabel("Parameter")
    plt.ylabel("Roc Auc Score")
    plt.tight_layout()
    plt.ylim(ymin=0.6)
    plt.legend(loc="best")
    plt.show()

### N estimators

In [ ]:
plot_validation_curve(x = x_train,
                      y = y_train.values.reshape(-1,),
                      modelo = RandomForestClassifier(class_weight= 'balanced' ,max_depth=5),
                      parametro = "n_estimators",
                      param_range = np.arange(50, 300, 50),
                      metrica = "roc_auc")

### Max_depth

In [ ]:
plot_validation_curve(x = x_train,
                      y = y_train.values.reshape(-1,),
                      modelo = RandomForestClassifier(class_weight= 'balanced'),
                      parametro = "max_depth",
                      param_range = np.arange(3, 10, 1),
                      metrica = "roc_auc")

### Min sample_split

In [ ]:
plot_validation_curve(x = x_train,
                      y = y_train.values.reshape(-1,),
                      modelo = RandomForestClassifier(class_weight= 'balanced' ,max_depth=5),
                      parametro = "min_samples_split",
                      param_range =  np.arange(0, 100, 15),
                      metrica = "roc_auc")

### Min samples leaf

In [ ]:
plot_validation_curve(x = x_train,
                      y = y_train.values.reshape(-1,),
                      modelo = RandomForestClassifier(class_weight= 'balanced' ,max_depth=5),
                      parametro = "min_samples_leaf",
                      param_range = np.arange(10, 150, 25),
                      metrica = "roc_auc")

### Hyperopt with Random forest exploring the best result of the validation curve output

- Com base no learning curve é possvel dimensionar melhor a busca de hiperparametros no hyperopt 

In [ ]:
from sklearn.model_selection import StratifiedKFold

def objective(params):
    params = {'n_estimators': int(params['n_estimators']), 'max_depth': int(params['max_depth'])}
    clf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', **params)
    score = cross_val_score(clf, x_test, y_test, scoring='roc_auc', cv=StratifiedKFold()).mean()
    print("ROC {:.3f} params {}".format(score, params))
    score = (1-score)
    return score

space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 25),
    'max_depth': hp.quniform('max_depth', 3, 8, 1),
    "max_features": ['sqrt','log2'],
    "min_samples_split" : hp.quniform('min_samples_split',10,200,10)
}

best_rf = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=5)

### Testing Smote Oversampling in a Random forest

In [ ]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import StratifiedKFold
def objective(params):
    params = {'n_estimators': int(params['n_estimators']), 'max_depth': int(params['max_depth'])}
    steps = [('over', SMOTE()), ('model', RandomForestClassifier(n_jobs=-1, **params))]
    clf = Pipeline(steps=steps)
    score = cross_val_score(clf, x_test, y_test, scoring='roc_auc', cv=StratifiedKFold()).mean()
    print("ROC {:.3f} params {}".format(score, params))
    score = (1-score)
    return score

space = {
    'n_estimators': hp.quniform('n_estimators', 50, 1000, 10),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    "max_features": ['sqrt','log2'],
    "min_samples_split" : hp.quniform('min_samples_split',50,200,10)
}

best_ = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=5)

print('best parametes', best)

### Testing a Boosting Algorithm (Xgboost with hiperparameter tuning hyperopt)

In [ ]:
import xgboost as xgb
xgb.set_config(verbosity=0)
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import StratifiedKFold
def objective(params):
    params = {'max_depth': int(params['max_depth'])
              , 'gamma': int(params['gamma'])
              , 'reg_alpha': int(params['reg_alpha'])
              , 'reg_lambda': int(params['reg_lambda'])
              , 'colsample_bytree': int(params['colsample_bytree'])
              , 'min_child_weight': int(params['min_child_weight'])
              , 'n_estimators': int(params['n_estimators'])
             }

    clf = xgb.XGBClassifier(n_jobs=-1,**params)
    score = cross_val_score(clf,  x_test, y_test, scoring='roc_auc', cv=StratifiedKFold()).mean()
    print("ROC {:.3f} params {}".format(score, params))
    score = (1-score)
    return score

space={'max_depth': hp.quniform("max_depth", 3, 8, 1),
        'gamma': hp.uniform ('gamma', 0,0.5),
        'reg_alpha' : hp.quniform('reg_alpha', 40,100,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 20, 1),
        'n_estimators': hp.quniform('n_estimators', 50, 500, 25),
    }

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=5)

print('best parametes', best)

### Training the best results from the Random forest

In [ ]:
print("Start training the best model")
model = RandomForestClassifier(n_jobs=-1
                                      , class_weight='balanced'
                                      , max_depth = int(best_rf['max_depth'])
                                      , min_samples_split = int(best_rf['min_samples_split'])
                                      , n_estimators = int(best_rf['n_estimators'])).fit(x_train,y_train)
print("Scoring the train data")
#Scoring the best model in train dataset
predict_train_entire = model.predict(x_train)
proba_train_entire = model.predict_proba(x_train)[:,1]

print("Scoring the test data")
#Scoring the best model in test dataset
predict_test_entire = model.predict(x_test)
proba_test_entire = model.predict_proba(x_test)[:,1]

print("Getting metrics")
# calculate scores
auc_train = roc_auc_score(y_train, proba_train_entire)
auc_test = roc_auc_score(y_test, proba_test_entire )

# calculate roc curves
fpr_train, tpr_train, _ = roc_curve(y_train, proba_train_entire)
fpr_test, tpr_test, _ = roc_curve(y_test, proba_test_entire)

# plot the roc curve for the model
pyplot.plot(fpr_train, tpr_train, linestyle='--', label='Train')
pyplot.plot(fpr_test, tpr_test, linestyle='--', label='Test')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

# summarize scores
print('ROC AUC Train =%.3f' % (auc_train))
print('ROC AUC Test =%.3f' % (auc_test))

print('-----------------------------------------------------')

ks_stat_train = binary_ks_curve(y_train, predict_train_entire)[3]
print('ks train =',ks_stat_train)

ks_stat_test = binary_ks_curve(y_test, predict_test_entire)[3]
print('ks test =',ks_stat_test)

print('-----------------------------------------------------')

recall_train = recall_score(y_train, predict_train_entire)
print('recall train =',recall_train)

recall_test = recall_score(y_test, predict_test_entire)
print('recall test =',recall_test)

print('-----------------------------------------------------')
print("------------------------")
print("Calculate decil in train")
print("------------------------")

avg_tgt = y_train.sum()/len(y_train)
df_data = x_train.copy()
X_data = df_data.copy()
df_data['Actual'] = y_train
df_data['Predict']= model.predict(X_data)
y_Prob = pd.DataFrame(model.predict_proba(X_data))
df_data['Prob_1']=list(y_Prob[1])
df_data.sort_values(by=['Prob_1'],ascending=False,inplace=True)
df_data.reset_index(drop=True,inplace=True)
df_data['Decile']=pd.qcut(df_data.index,5,labels=False)
output_df = pd.DataFrame()
grouped = df_data.groupby('Decile',as_index=False)
output_df['Qtd']=grouped.count().Actual
output_df['Sum_Target']=grouped.sum().Actual
output_df['Per_Target'] = (output_df['Sum_Target']/output_df['Sum_Target'].sum())*100
output_df['Per_Acum_Target'] = output_df.Per_Target.cumsum()
output_df['Max_proba']=grouped.max().Prob_1
output_df['Min_proba']=grouped.min().Prob_1
output_df["Per_Pop"] = (output_df["Qtd"]/len(y_train))*100
output_df["Lift"] = output_df["Per_Acum_Target"]/output_df.Per_Pop.cumsum()
output_df= output_df.drop(columns='Per_Pop')
print(round(output_df,3))

print("------------------------")
print("Calculate decil in test")
print("------------------------")
Avg_tgt = y_test.sum()/len(y_test)
df_data = x_test.copy()
X_data = df_data.copy()
df_data['Actual'] = y_test
df_data['Predict']= model.predict(X_data)
y_Prob = pd.DataFrame(model.predict_proba(X_data))
df_data['Prob_1']=list(y_Prob[1])
df_data.sort_values(by=['Prob_1'],ascending=False,inplace=True)
df_data.reset_index(drop=True,inplace=True)
df_data['Decile']=pd.qcut(df_data.index,5,labels=False)
output_df = pd.DataFrame()
grouped = df_data.groupby('Decile',as_index=False)
output_df['Qtd']=grouped.count().Actual
output_df['Sum_Target']=grouped.sum().Actual
output_df['Per_Target'] = (output_df['Sum_Target']/output_df['Sum_Target'].sum())*100
output_df['Per_Acum_Target'] = output_df.Per_Target.cumsum()
output_df['Max_proba']=grouped.max().Prob_1
output_df['Min_proba']=grouped.min().Prob_1
output_df["Per_Pop"] = (output_df["Qtd"]/len(y_test))*100
output_df["Lift"] = output_df["Per_Acum_Target"]/output_df.Per_Pop.cumsum()
output_df= output_df.drop(columns='Per_Pop')
print(round(output_df,3))

### Showing the best discrimination threshould

In [ ]:
from yellowbrick.classifier import discrimination_threshold
visualizer =  discrimination_threshold(model,X = x_train,y = y_train)

### Print the error of the model

In [ ]:
from yellowbrick.classifier import ClassPredictionError

# Instantiate the classification model and visualizer
visualizer = ClassPredictionError(
    estimator = model
)

# Fit the training data to the visualizer
visualizer.fit(x_train, y_train['TARGET'])

# Evaluate the model on the test data
visualizer.score(x_test, y_test['TARGET'])

# Draw visualization
visualizer.show()

### Showing the features importance based ina shape values

In [ ]:
x_test_shap = x_test.sample(500)
explainer = shap.Explainer(model.predict, x_test_shap)
shap_values = explainer(x_test_shap)
shap.plots.bar(shap_values)

### Summary plot

In [ ]:
shap.summary_plot(shap_values)

### Make a submission

In [ ]:
# making predctions on the test dataset (df_test), from Kaggle, with the selected features and optimized parameters
colunas = x_test.columns
df_teste = pd.read_csv("../input/santander-customer-satisfaction/test.csv").filter(colunas)

proba_test = model.predict_proba(df_teste)[:,1]
# saving the result into a csv file to be uploaded into Kaggle late subimission 
# https://www.kaggle.com/c/santander-customer-satisfaction/submit
sub = pd.Series(proba_test, index = df_test['ID'], 
name = 'TARGET')
sub.to_csv('data/df_test_predictions.csv')